## 20221209

In [ ]:
# import

from collections import defaultdict, namedtuple
from htbuilder import div, big, h2, styles
from htbuilder.units import rem
from math import floor
import altair as alt
import datetime
import functools
import numpy as np
import pandas as pd
import re
import streamlit as st
import time
import plotly.express as px
from vega_datasets import data
from st_aggrid import AgGrid

from streamlit_app import instruction as instr
from streamlit_app import form as form
from streamlit_app import custom_display as cd

import altair_viewer
alt.renderers.enable('altair_viewer', inline=True)
# alt.renderers.enable('notebook')
# alt.renderers.enable('mimetype')

In [ ]:
COLOR_RED = "#ED2E24"
COLOR_DARK_BLUE = "#052A5F"
COLOR_BLUE = "#0476BF"

### Baseline MA

In [ ]:
# MA 5-10 chart

path = 'models-v2-lle/testset_w_lle.csv'
df = pd.read_csv(path)
df.head()

In [ ]:
home_team_name = 'Arizona Diamondbacks (ARI)'
# extract
home_team_name = home_team_name.split(' (')[1].replace(')', '')
home_team_name

In [ ]:
home_team_name = "ARI"

df_home = df[df["team2_name"] == home_team_name]
df_home.head(3)

In [ ]:
# baseline value 

# given date and team name 

def nearest(items, pivot):
    target_date = pd.to_datetime(min([i for i in items if i <= pivot], key=lambda x: abs(x - pivot)))
    return target_date


date_str = np.datetime64('2022-04-20')
team_name = "ARI"

# get baseline value
baseline_value = df_home[(df_home["date"] == nearest(df_home['date'].values, date_str)) & (df_home["team2_name"] == team_name)]["previous_5_to_10MA"].values[0]
baseline_value

In [ ]:
df_home["date"] = pd.to_datetime(df_home["start_time"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").date()))
df_home["date"]

In [ ]:
source = df_home

bar = alt.Chart(source).mark_bar(color=COLOR_BLUE).encode(  # type: ignore
    x = alt.X('date:O', axis=alt.Axis(title='Date', labelAngle=30)), # type: ignore
    y = alt.Y('attendance:Q', axis=alt.Axis(title='Attendance')), # type: ignore
    tooltip=[
        alt.Tooltip('attendance', title='Attendance'), # type: ignore
        alt.Tooltip('previous_5_to_10MA', title='Previous 5-10 MA'), # type: ignore
        alt.Tooltip('date', title='Date'), # type: ignore
    ]
)

line = alt.Chart(source).mark_line(color=COLOR_RED).encode( # type: ignore
    x = 'date:T',
    y = 'previous_5_to_10MA:Q',
)


tmp_plt = (bar + line).interactive()
altair_viewer.display(tmp_plt, inline=True)

In [ ]:
source = data.wheat()

bar = alt.Chart(source).mark_bar(color=COLOR_BLUE).encode(  # type: ignore
    x='year:O',
    y='wheat:Q',
    tooltip=['year', 'wheat']
)

line = alt.Chart(source).mark_line(color=COLOR_RED).transform_window(  # type: ignore
    # The field to average
    rolling_mean='mean(wheat)',
    # The number of values before and after the current value to include.
    frame=[-9, 0]
).encode(
    x='year:O',
    y='rolling_mean:Q',
    tooltip=['year']
)

tmp_plt = (bar + line).interactive()
altair_viewer.display(tmp_plt, inline=True)

### Feature Importance

In [ ]:
path = 'streamlit_app/instruction/model_feature_importance.xlsx'
summary_df = pd.read_excel(path, sheet_name='Summary', engine='openpyxl')
raw_df = pd.read_excel(path, sheet_name='Ft. Importance Table', engine='openpyxl')

In [ ]:
raw_df.head()

In [ ]:
model_df = raw_df[raw_df["Model"] == "XGBoost"]

In [ ]:
source = model_df

tmp_plt = alt.Chart(source).mark_bar(color=COLOR_BLUE).encode(  # type: ignore
    x='Metric:Q',
    y=alt.Y('Processed Feature Name:N', sort='-x', axis=alt.Axis(labelLimit=100)), # type: ignore
    tooltip=[
        'Model',
        'Metric', 
             alt.Tooltip('Processed Feature Name', title='Feature Name'), # type: ignore
             alt.Tooltip('Feature', title='Raw Feature Name'), # type: ignore
             
            ]
)

# st.altair_chart(tmp_plt, use_container_width=True)
altair_viewer.display(tmp_plt, inline=True)

## Before 20221209

In [ ]:
import pandas as pd
data = pd.read_pickle('testset.pickle')

In [ ]:
# print first 5 data keys
print(len(list(data.keys())))

In [ ]:
list(list(data.keys())[2450:2455])

In [ ]:
data['https://www.baseball-reference.com/boxes/PHI/PHI202210150.shtml']